# 2 - eBay Data Preprocessing

### Workflow for Preprocessing
- Create average hash of images to detect duplicates.
- Calculate an image "uniqueness" score based on frequency of duplicates. 
- Standardize prices based on items MSRP. 
- Calculate a description uniqueness score based on item's description.

In [1]:
import imagehash
import mysql.connector
import os
import pandas as pd
from PIL import Image
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import cosine_distances

## Creating Image Hashes  

One of our hypotheses in this study is that the uniqueness of information 
will affect the final sale price. Thus, an average hash of the image is 
created that allows use to programmatically identify duplicates.  

In [2]:
def connect_to_mysql(user, pw, host, database):
    '''Creates connection with mysql database and returns cnx & cursor.

    Parameters
    ----------
    user : str
        MySQL user name.
    pw : str
        MySQL password. 
    host : str
        MySQL hostname or IP address. 
    database : str
        MySQL database name. 

    Returns
    -------
    cnx 
        MySQL connection object.
    cursor
        MySQL cursor object.    
    '''
    cnx = mysql.connector.connect(user=user, password=pw,
                                  host=host, database=database)
    cursor = cnx.cursor()
    return cnx, cursor


def retrieve_from_mysql(query, user, pw, host, db):
    """Retrieves data from MySQL database.  

    Parameters
    ----------
    query : str 
        Valid MySQL query.  
    user : str
        MySQL user name.
    pw : str
        MySQL password. 
    host : str
        MySQL hostname or IP address. 
    database : str
        MySQL database name.

    Returns
    -------
    data : lst of tuples 
        Query results in list of tuples. 
    """
    cnx, cursor = connect_to_mysql(user, pw, host, db)

    cursor.execute(query)

    data = cursor.fetchall()
    cnx.close()
    return data


def read_config_file(file_name):
    '''Takes json file name as str, returns dict of file contents.
       Used in Python2 to convert unicode json to ascii. 

    Parameters
    ----------
    file_name : str
        Json filename.        

    Returns
    -------
    dict 
        A dictionary of with str in ASCII.  
    '''

    def ascii_encode_dict(data):
        ''' https://stackoverflow.com/q/9590382  
        '''

        def ascii_encode(x): return x.encode(
            'ascii') if isinstance(x, unicode) else x
        return dict(map(ascii_encode, pair) for pair in data.items())

    with open(file_name, 'r') as f:
        config = json.load(f)

    return json.loads(json.dumps(config), object_hook=ascii_encode_dict)


def add_hashes_to_mysql(data):
    """Adds hashed image data to MySQL database.   

    Parameters
    ----------
    data : dict 
        Dict of data from hash_dict() and dimensions() functions. 
    Returns
    -------
    None 
    """

    add_imagehash = ("INSERT INTO image_hash"
                     "(itemId, hash, mirror, left_hash, right_hash, width, height)"
                     "VALUES (%s, %s, %s, %s, %s, %s, %s)")

    imagehash_data = (data['itemId'],
                      str(data['hash']),
                      str(data['mirror']),
                      str(data['left']),
                      str(data['right']),
                      data['width'],
                      data['height'])

    try:
        cursor.execute(add_imagehash, imagehash_data)
        cnx.commit()

    except:
        print "Failed to add hash for item {}.".format(data['itemId'])
        print '-----------------------'
        print ''


def hash_dict(im):
    """Returns average hash for different orientations. 
       Requires pillow, imagehash.   

    Parameters
    ----------
    im : str or Image object 
        Path to file or a pil image object. 

    Returns
    -------
    hashes: dict 
        Dictionary of average hashes of images with variations. 
        Generate hashes for the image, including variations of the image:
            * hash: Regular image
            * mirror: Mirrored (left-right)
            * left: Rotated left (90deg)
            * right: Rotated right (270deg)
    """
    # Open image if not PIL image object.
    if not isinstance(im, Image.Image):
        im = Image.open(im)

    # Resize, downsample, and convert to greyscale.
    im = im.resize((16, 16), Image.ANTIALIAS).convert('L')

    hashes = {}

    # Regular hash
    hashes['hash'] = imagehash.average_hash(im)

    # Mirror hash
    mirror_im = im.transpose(Image.FLIP_LEFT_RIGHT)
    hashes['mirror'] = imagehash.average_hash(mirror_im)

    # Rotated 90deg hash
    left_im = im.transpose(Image.ROTATE_90)
    hashes['left'] = imagehash.average_hash(left_im)

    # Rotated 270deg hash
    right_im = im.transpose(Image.ROTATE_270)
    hashes['right'] = imagehash.average_hash(right_im)

    del im
    return hashes


def dimensions(im):
    """Returns image dimensions (helper). 

    Parameters
    ----------
    im : str or Image object 
        Path to file or a pil image object. 

    Returns
    -------
    hashes: dict
        Dict of image dimensions. 
            * width: image width
            * height: image height 
    """
    if not isinstance(im, Image.Image):
        im = Image.open(im)

    dims = {}
    dims['width'] = im.size[0]
    dims['height'] = im.size[1]
    del im
    return dims

In [ ]:
# Example usage of above functions.

if __name__ == "__main__":

    # Read config for credentials.
    secrets = read_config_file('secrets.json')

    # Initiate mysql connection.
    cnx, cursor = connect_to_mysql(secrets['user'], secrets['password'],
                                   secrets['host'], secrets['database'])

    # Create a list of image files to hash.  Assumes running from same
    # folder as notebook 1.
    dir_path = os.path.join(os.getcwd(), 'ebay_listing_photos')

    image_list = [file for file in os.listdir(dir_path)
                  if file.lower().endswith((".jpg", ".png", ".bmp"))]

    # Don't hash previously hashed images.
    query = "SELECT DISTINCT(itemId) from image_hash"
    prev_hashed = [i[0] for i in retrieve_from_mysql(query,
                                                     secrets['user'],
                                                     secrets['password'],
                                                     secrets['host'],
                                                     secrets['database'])]

    new_images = [image for image in image_list
                  if image.split('.')[0] not in prev_hashed]

    # Hash images and add to MySQL.
    for image in new_images:
        image_path = os.path.join(dir_path, image)
        image_data['itemId'] = image.split('.')[0]
        image_data.update(hash_dict(image_path))
        image_data.update(dimensions(image_path))
        add_hashes_to_mysql(image_data)

    cnx.close()

## Calculating Image Uniqueness 

Now that we've hashed our images we can now identify duplicates based on the hash values.  Although it's possible to make this as complex as we like (e.g. comparing various orientations, calculating hamming distances for near matches) we're simply going to look for exact duplicates with the original image orientation. After all, most listings for a product should have *fairly* similar images.  

Finally, the raw counts would be misleading as some products appear more frequently in the dataset than others.  Therefore, we'll divide the image frequency by the number of times its respective product appeared in the dataset to normalize the scores. 

In [4]:
# Get hash, name, and itemId from MySQL.
cnx, _ = connect_to_mysql(secrets['user'], secrets['password'],
                          secrets['host'], secrets['database'])

query = "SELECT item_finding_api.itemId, item_finding_api.name, " \
        "image_hash.hash " \
        "from item_finding_api " \
        "Right JOIN image_hash " \
        "ON (item_finding_api.itemId=image_hash.itemId);"

df = pd.read_sql(query, con=cnx)
cnx.close()

# Calculate image uniqueness.
df2 = df.groupby(['name', 'hash']).size().reset_index(name='image_count')
a = df2.groupby('name')['image_count'].transform('sum')
df2['image_uniqueness'] = 1 - df2['image_count'].div(a)
df2.drop('name', axis=1, inplace=True)
df_hash = pd.merge(df, df2, on='hash', how='left')
df_hash.drop('name', axis=1, inplace=True)

del df, df2

## Standardizing Prices 

When examining a diverse set of products it is not meaningful to calculate an average price across all products in the dataset.  Instead, we'll use the method described by [Dimoka, Pavlou, and Hong](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=1976541) to calculate a price premium by subtracting the items MSRP from the Final Price and then dividing it by the MSRP and multiplying it by -1 to create a standardized value representing the percentage of the final selling price below the MSRP: 

\begin{equation*}
Discount = \frac{(Price-MSRP)}{(MSRP)}* -1 
\end{equation*}

The MSRP for each product was determined by visiting the website of the manufacturer, publisher, or studio and noting their suggested retail prices. MSRPs are appropriate benchmark prices for this type of calculation because although products are often not sold at the MSRP, their purpose is to serve as an anchoring price to prevent retailers from over or underpricing items for sale.

In [5]:
msrp = {'GTA 5 - Xbox': 59.99,
        'windows 7 professional': 133.99,
        'Bose Quietcomfort 15': 299.95,
        'fifty shades of grey trilogy': 47.85,
        'rayban wayfarer 2140 901': 155.00,
        'Avengers - DVD': 17.95,
        'WD Blue HDD': 54.99,
        'Coco Mademoiselle (Eau de Parfum)': 122.00
        }

cnx, _ = connect_to_mysql(secrets['user'], secrets['password'],
                          secrets['host'], secrets['database'])

query = 'SELECT itemId, name, saleAmount from item_finding_api '

df_disc = pd.read_sql(query, con=cnx)
cnx.close()

df_disc['msrp'] = df_disc['name'].map(msrp)

df_disc['discount'] = ((df_disc['saleAmount'] - df_disc['msrp'])
                       / df_disc['msrp'])*-1

## Search vs. Experience Goods 

We're going to quickly map the search and experience good categories to our 
product listings.  

In [6]:
search_exp = {'GTA 5 - Xbox': 'search',
              'windows 7 professional': 'search',
              'Bose Quietcomfort 15': 'experience',
              'fifty shades of grey trilogy': 'search',
              'rayban wayfarer 2140 901': 'experience',
              'Avengers - DVD': 'search',
              'WD Blue HDD': 'search',
              'Coco Mademoiselle (Eau de Parfum)': 'experience'
              }

df_disc['search_exp'] = df_disc['name'].map(search_exp)

## Calculating Description Uniqueness 
Description uniqueness represents the degree to which an item listings textual description was similar or different from other descriptions for the same product. First, for each product a matrix of cosine similarity scores was calculated by comparing the descriptions for each pair of listings. Cosine similarity is a metric used in information retrieval that allows for the comparison of vectors representing the frequency with which terms appear in a document. Given two vectors x and y,

\begin{equation}
\cos ({\bf x},{\bf y})= {{\bf x} {\bf y} \over \|{\bf x}\| \|{\bf y}\|}
\end{equation}

The cosine similarities were further converted into distance or dissimilarity scores as follows:

\begin{equation}
\ dist ({\bf x},{\bf y})= 1-cos ({\bf x},{\bf y})
\end{equation}

The resulting score represents the degree to which a pair of listings were similar. Values that equal 0 indicated that the compared descriptions were identical while values approaching 1 indicated that the descriptions were quite different. Next, the matrix rows were averaged to create an aggregate uniqueness score representing how much a particular item description resembled all other item descriptions for the same product. Listings that did not include a text description, and thus resulted in a null value (N=232), were excluded when calculating the matrix row averages.

In [7]:
# Retrieve descriptions from MySQL
cnx, cursor = connect_to_mysql(secrets['user'], secrets['password'],
                               secrets['host'], secrets['database'])

query = "SELECT item_shopping_api.itemId, item_finding_api.name, " \
        "item_shopping_api.Description " \
        "from item_shopping_api " \
        "LEFT JOIN item_finding_api " \
        "ON (item_shopping_api.itemId = item_finding_api.itemId) " \
        "WHERE item_shopping_api.Description is not Null"

data = retrieve_from_mysql(query, secrets['user'], secrets['password'],
                           secrets['host'], secrets['database'])

cnx.close()

# Sort by product prior to calculating description uniqueness.
sorted_by_product = {}

for listing in data:
    if listing[1] not in sorted_by_product:
        sorted_by_product[listing[1]] = [listing]
    else:
        sorted_by_product[listing[1]].append(listing)

# Calculate description uniqueness score by product.
vectorizer = TfidfVectorizer()

uniqueness = []

for key, value in sorted_by_product.iteritems():
    # Create an index of itemIds.
    ids = []
    descriptions = []
    for i in value:
        ids.append(i[0]), descriptions.append(i[2])

    tfidf = TfidfVectorizer().fit_transform(descriptions)
    avg_cos_dist = cosine_distances(tfidf).mean(axis=0)
    uniqueness += zip(ids, avg_cos_dist)

df_u = pd.DataFrame(uniqueness, columns=['itemId', 'text_uniqueness'])

## Joining Data and Writing to File 

Finally, we're just going to do a bit of cleanup.  We're going to grab a few more interesting variables from MySQL, join them with our new calculated variables, and then we'll write the data to file for analysis in another notebook. 

In [8]:
cnx, _ = connect_to_mysql(secrets['user'], secrets['password'],
                          secrets['host'], secrets['database'])

query = "Select item_finding_api.itemId, item_finding_api.bidCount, " \
        "listing.startTime, listing.endTime, item_shopping_api.HitCount, " \
        "seller.feedbackScore, content_coding.content_coding " \
        "from item_finding_api " \
        "LEFT JOIN listing " \
        "ON (item_finding_api.itemId = listing.itemId) " \
        "LEFT JOIN item_shopping_api " \
        "ON (item_finding_api.itemId = item_shopping_api.itemId) " \
        "LEFT JOIN seller " \
        "ON (item_finding_api.itemId = seller.itemId) " \
        "LEFT JOIN content_coding " \
        "ON (item_finding_api.itemId = content_coding.itemId) " \
        "WHERE item_shopping_api.Description is not Null " \
        "AND content_coding.content_coding <> '' " \
        "AND content_coding.content_coding <> 'No Photo Provided';"

df = pd.read_sql(query, con=cnx)
cnx.close()

dfs = [df, df_u, df_hash, df_disc]
df_final = reduce(lambda left, right: pd.merge(left, right, on='itemId'), dfs)
df_final.to_csv('ebay_data.csv', header=True)